In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

In [2]:
#scrape a single game shots. We will allow users to enter a base url
base_url = 'https://understat.com/match/'
match = str(input('Please enter the match id: '))
url = base_url+match

Please enter the match id: 19021


In [5]:
#Use requests to get the webpage and BeautifulSoup to parse the page
res = requests.get(url)
soup = BeautifulSoup(res.content, 'lxml')
scripts = soup.find_all('script')

In [6]:
#get only the shotsData
# strings = scripts[1].string

In [14]:
strings = scripts[1].string

In [15]:
strings

"\n\tvar shotsData \t= JSON.parse('\\x7B\\x22h\\x22\\x3A\\x5B\\x7B\\x22id\\x22\\x3A\\x22489675\\x22,\\x22minute\\x22\\x3A\\x225\\x22,\\x22result\\x22\\x3A\\x22MissedShots\\x22,\\x22X\\x22\\x3A\\x220.919000015258789\\x22,\\x22Y\\x22\\x3A\\x220.43599998474121093\\x22,\\x22xG\\x22\\x3A\\x220.07861322909593582\\x22,\\x22player\\x22\\x3A\\x22Felipe\\x22,\\x22h_a\\x22\\x3A\\x22h\\x22,\\x22player_id\\x22\\x3A\\x227921\\x22,\\x22situation\\x22\\x3A\\x22SetPiece\\x22,\\x22season\\x22\\x3A\\x222022\\x22,\\x22shotType\\x22\\x3A\\x22Head\\x22,\\x22match_id\\x22\\x3A\\x2219021\\x22,\\x22h_team\\x22\\x3A\\x22Atletico\\x20Madrid\\x22,\\x22a_team\\x22\\x3A\\x22Real\\x20Madrid\\x22,\\x22h_goals\\x22\\x3A\\x221\\x22,\\x22a_goals\\x22\\x3A\\x222\\x22,\\x22date\\x22\\x3A\\x222022\\x2D09\\x2D18\\x2019\\x3A00\\x3A00\\x22,\\x22player_assisted\\x22\\x3A\\x22Rodrigo\\x20de\\x20Paul\\x22,\\x22lastAction\\x22\\x3A\\x22Chipped\\x22\\x7D,\\x7B\\x22id\\x22\\x3A\\x22489676\\x22,\\x22minute\\x22\\x3A\\x2211\\x22,\\x2

In [16]:
# strip unnecessary symbols and get only JSON data 
ind_start = strings.index("('")+2
ind_end = strings.index("')") 
json_data = strings[ind_start:ind_end] 
json_data = json_data.encode('utf8').decode('unicode_escape')

#convert string to json format
data = json.loads(json_data)

In [17]:
data

{'h': [{'id': '489675',
   'minute': '5',
   'result': 'MissedShots',
   'X': '0.919000015258789',
   'Y': '0.43599998474121093',
   'xG': '0.07861322909593582',
   'player': 'Felipe',
   'h_a': 'h',
   'player_id': '7921',
   'situation': 'SetPiece',
   'season': '2022',
   'shotType': 'Head',
   'match_id': '19021',
   'h_team': 'Atletico Madrid',
   'a_team': 'Real Madrid',
   'h_goals': '1',
   'a_goals': '2',
   'date': '2022-09-18 19:00:00',
   'player_assisted': 'Rodrigo de Paul',
   'lastAction': 'Chipped'},
  {'id': '489676',
   'minute': '11',
   'result': 'BlockedShot',
   'X': '0.8730000305175781',
   'Y': '0.325',
   'xG': '0.040581703186035156',
   'player': 'Yannick Carrasco',
   'h_a': 'h',
   'player_id': '2272',
   'situation': 'OpenPlay',
   'season': '2022',
   'shotType': 'RightFoot',
   'match_id': '19021',
   'h_team': 'Atletico Madrid',
   'a_team': 'Real Madrid',
   'h_goals': '1',
   'a_goals': '2',
   'date': '2022-09-18 19:00:00',
   'player_assisted': 'Koke

In [18]:
x = []
y = []
xG = []
result = []
team = []
data_away = data['a']
data_home = data['h']

for index in range(len(data_home)):
    for key in data_home[index]:
        if key == 'X':
            x.append(data_home[index][key])
        if key == 'Y':
            y.append(data_home[index][key])
        if key == 'h_team':
            team.append(data_home[index][key])
        if key == 'xG':
            xG.append(data_home[index][key])
        if key == 'result':
            result.append(data_home[index][key])

for index in range(len(data_away)):
    for key in data_away[index]:
        if key == 'X':
            x.append(data_away[index][key])
        if key == 'Y':
            y.append(data_away[index][key])
        if key == 'a_team':
            team.append(data_away[index][key])
        if key == 'xG':
            xG.append(data_away[index][key])
        if key == 'result':
            result.append(data_away[index][key])

In [19]:
col_names = ['x','y','xG','result','team']
df = pd.DataFrame([x,y,xG,result,team],index=col_names)
df = df.T

In [20]:
df

,x,y,xG,result,team
0,0.919000015258789,0.43599998474121093,0.07861322909593582,MissedShots,Atletico Madrid
1,0.8730000305175781,0.325,0.040581703186035156,BlockedShot,Atletico Madrid
2,0.9130000305175782,0.5890000152587891,0.0839962363243103,BlockedShot,Atletico Madrid
3,0.6719999694824219,0.48,0.012301268987357616,MissedShots,Atletico Madrid
4,0.8030000305175782,0.46,0.06537076830863953,SavedShot,Atletico Madrid
5,0.9269999694824219,0.4540000152587891,0.1079680323600769,MissedShots,Atletico Madrid
6,0.9290000152587891,0.7030000305175781,0.04961972311139107,SavedShot,Atletico Madrid
7,0.71,0.4420000076293945,0.014143135398626328,SavedShot,Atletico Madrid
8,0.749000015258789,0.4420000076293945,0.02170601859688759,BlockedShot,Atletico Madrid
9,0.9080000305175782,0.255,0.03163771703839302,MissedShots,Atletico Madrid
